In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = [224, 224]

train_path = 'image/Training'
valid_path = 'image/Testing'

In [3]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [4]:

for layer in vgg.layers:
  layer.trainable = False



In [5]:
folders = glob('image/Training/*')

In [6]:
x = Flatten()(vgg.output)

In [7]:
prediction = Dense(len(folders), activation='softmax')(x)

In [8]:
model = Model(inputs=vgg.input, outputs=prediction)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)



In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('image/Training',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('image/Testing',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')



Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [13]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/Users/glipne/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
90/90 [==============================] - 979s 11s/step - loss: 0.3004 - accuracy: 0.8861 - val_loss: 1.5568 - val_accuracy: 0.7183
Epoch 2/5
90/90 [==============================] - 939s 10s/step - loss: 0.2650 - accuracy: 0.9021 - val_loss: 1.5323 - val_accuracy: 0.7310
Epoch 3/5
90/90 [==============================] - 932s 10s/step - loss: 0.2372 - accuracy: 0.9098 - val_loss: 1.4875 - val_accuracy: 0.7157
Epoch 4/5
90/90 [==============================] - 1115s 12s/step - loss: 0.2119 - accuracy: 0.9230 - val_loss: 1.5653 - val_accuracy: 0.7487
Epoch 5/5
90/90 [==============================] - 1005s 11s/step - loss: 0.1956 - accuracy: 0.9275 - val_loss: 1.5956 - val_accuracy: 0.7411


In [18]:
import numpy as np

#from google.colab import files
from keras.preprocessing import image

#uploaded=files.upload()

#for fn in uploaded.keys():
 
  # predicting images
path='/Users/glipne/Desktop/Python/AI:ML/brain_tumer_detection/trial/p (8).jpg'
img=image.load_img(path, target_size=(224,224))
  
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
  
classes = model.predict(images)
for cls in training_set.class_indices:
    print(cls+ " : "+str(classes[0][training_set.class_indices[cls]]))
    

glioma_tumor : 0.0
meningioma_tumor : 1.0
no_tumor : 1.5139982e-13
pituitary_tumor : 0.0


In [ ]:
# plot the loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')